In [1]:
import json
import os
import numpy as np
import glob

In [2]:
with open('annotations/instances_val2014.json') as f:
    data = json.load(f)

In [3]:
images = {}
for i in data['images']:
    try:
        images[i['id']] = {'name': i['file_name'], 'height': i['height'], 'width': i['width']}
    except:
        pass

In [18]:
persons = {}
for i in data['annotations']:
    try:
        if i['category_id'] == 1:
            if i['image_id'] not in persons.keys():
                persons[i['image_id']] = np.array(i['segmentation'][0])
            else:
                if type(persons[i['image_id']]) == type(np.array([])):
                    persons[i['image_id']] = [persons[i['image_id']], np.array(i['segmentation'][0])]
                else:
                    persons[i['image_id']].append(np.array(i['segmentation'][0]))
    except:
        pass

In [19]:
if not os.path.exists('labels'):
    os.mkdir('labels')

for i in persons.keys():
    with open(f"labels/{images[i]['name'][:-4]+'.txt'}", 'w') as f:
        if type(persons[i]) == type([]):
            for j in persons[i]:
                j[::2] /= images[i]['width']
                j[1::2] /= images[i]['height']
                f.write('0 '+' '.join(map(lambda x: str(x), j)))
        else:
            persons[i][::2] /= images[i]['width']
            persons[i][1::2] /= images[i]['height']
            f.write('0 '+' '.join(map(lambda x: str(x), persons[i])))

### Delete images which not person in it

In [139]:
images = set(map(lambda x: x[16:-4],glob.glob('datasets/images/*')))
labels = set(map(lambda x: x[16:-4],glob.glob('datasets/labels/*')))
len(images), len(labels)

(40504, 21634)

In [142]:
for i in list(images-labels):
    os.remove(f'datasets/images/{i}.jpg')

In [143]:
images = set(map(lambda x: x[15:-4],glob.glob('datasets/images/*')))
labels = set(map(lambda x: x[15:-4],glob.glob('datasets/labels/*')))
len(images), len(labels)

(21634, 21634)

### Split dataset to train and validation

In [154]:
import shutil

images = glob.glob('datasets/train/images/*')
labels = glob.glob('datasets/train/labels/*')
images.sort()
labels.sort()

In [150]:
images[:10]

['datasets/train/images/COCO_val2014_000000000074.jpg',
 'datasets/train/images/COCO_val2014_000000000136.jpg',
 'datasets/train/images/COCO_val2014_000000000139.jpg',
 'datasets/train/images/COCO_val2014_000000000192.jpg',
 'datasets/train/images/COCO_val2014_000000000241.jpg',
 'datasets/train/images/COCO_val2014_000000000257.jpg',
 'datasets/train/images/COCO_val2014_000000000294.jpg',
 'datasets/train/images/COCO_val2014_000000000328.jpg',
 'datasets/train/images/COCO_val2014_000000000338.jpg',
 'datasets/train/images/COCO_val2014_000000000357.jpg']

In [151]:
labels[:10]

['datasets/train/labels/COCO_val2014_000000000074.txt',
 'datasets/train/labels/COCO_val2014_000000000136.txt',
 'datasets/train/labels/COCO_val2014_000000000139.txt',
 'datasets/train/labels/COCO_val2014_000000000192.txt',
 'datasets/train/labels/COCO_val2014_000000000241.txt',
 'datasets/train/labels/COCO_val2014_000000000257.txt',
 'datasets/train/labels/COCO_val2014_000000000294.txt',
 'datasets/train/labels/COCO_val2014_000000000328.txt',
 'datasets/train/labels/COCO_val2014_000000000338.txt',
 'datasets/train/labels/COCO_val2014_000000000357.txt']

In [158]:
for i in range(5000):
    os.rename(images[i], images[i].replace('train', 'val'))
    os.rename(labels[i], labels[i].replace('train', 'val'))